# MANUAL NET ARCHITECTURE

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import glob
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from PIL import Image

from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
bacterial_leaf_blight = 'DATASET/TRAIN/bacterial_leaf_blight'
bacterial_leaf_streak = 'DATASET/TRAIN/bacterial_leaf_streak'
bacterial_panicle_blight = 'DATASET/TRAIN/bacterial_panicle_blight'
blast = 'DATASET/TRAIN/blast'
brown_spot = 'DATASET/TRAIN/brown_spot'
dead_heart = 'DATASET/TRAIN/dead_heart'
downy_mildew = 'DATASET/TRAIN/downy_mildew'
hispa = 'DATASET/TRAIN/hispa'
normal = 'DATASET/TRAIN/normal'
tungro = 'DATASET/TRAIN/tungro'

In [ ]:
def plot_images(item_dir, n=6):
    all_item_dir = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dir][:n]
    
    plt.figure(figsize=(80, 40))
    for idx, img_path in enumerate(item_files):
        plt.subplot(3, n, idx+1)
        img = plt.imread(img_path)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
    
    plt.tight_layout()
    
def image_details_print(data,path):
    print('======== Images in: ', path)
    for key,values in data.items():
        print(key,':\t', values)


def images_details(path):
    files=[f for f in glob.glob(path + "**/*.*", recursive=True)]
    data={}
    data['Images_count']=len(files)
    data['Min_width']=10**100
    data['Max_width']=0
    data['Min_height']=10**100
    data['Max_height']=0
    
    for f in files:
        img=Image.open(f)
        width,height=img.size
        data['Min_width']=min(width,data['Min_width'])
        data['Max_width']=max(width, data['Max_width'])
        data['Min_height']=min(height, data['Min_height'])
        data['Max_height']=max(height, data['Max_height'])
        
    image_details_print(data,path)

In [ ]:
print("")
print("TRAINING DATA FOR bacterial_leaf_blight:")
print("")
images_details(bacterial_leaf_blight)
print("")
plot_images(bacterial_leaf_blight, 10) 

In [ ]:
print("")
print("TRAINING DATA FOR bacterial_leaf_streak:")
print("")
images_details(bacterial_leaf_streak)
print("")
plot_images(bacterial_leaf_streak, 10)

In [ ]:
print("")
print("TRAINING DATA FOR bacterial_panicle_blight:")
print("")
images_details(bacterial_panicle_blight)
print("")
plot_images(bacterial_panicle_blight, 10)

In [ ]:
print("")
print("TRAINING DATA FOR hispa:")
print("")
images_details(hispa)
print("")
plot_images(hispa, 10)

In [ ]:
print("")
print("TRAINING DATA FOR dead_heart:")
print("")
images_details(dead_heart)
print("")
plot_images(dead_heart, 10)

In [ ]:
print("")
print("TRAINING DATA FOR tungro:")
print("")
images_details(tungro)
print("")
plot_images(tungro, 10)

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
training_set=train_datagen.flow_from_directory('dataset/train',target_size=(224,224),batch_size=32,class_mode='categorical')

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set=test_datagen.flow_from_directory('dataset/test',target_size=(224,224),batch_size=32,class_mode='categorical')

In [ ]:
Classifier=Sequential()
Classifier.add(Convolution2D(32,(3,3),input_shape=(224,224,3),activation='relu'))
Classifier.add(MaxPooling2D(pool_size=(2,2)))
Classifier.add(Flatten())
Classifier.add(Dense(38, activation='relu'))
Classifier.add(Dense(10, activation='softmax'))
Classifier.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model_path = "MANUAL.h5"

callbacks = [
    ModelCheckpoint(model_path, monitor='accuracy', verbose=1, save_best_only=True)
]

In [ ]:
epochs = 10
batch_size = 512

In [ ]:
#### Fitting the model
history = Classifier.fit(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, 
           validation_data=test_set,validation_steps=test_set.samples // batch_size,
           callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

def graph():
    #Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

graph()

In [ ]:
import matplotlib.pyplot as plt

def graph():
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
graph()